# Artificial  gappy data

We'll have a noisy gappy sine curve and see what `zfp` can do.

In [ ]:
import zfpy
import numpy as np
import pandas as pd
import hvplot.pandas

In [ ]:
N = 1000
noise_level = 0.3
signal_level = 1.0
signal_periods = 10
random_seed = 12345
fraction_valid = 0.5
zfpy_kwargs = {"tolerance": 1e-5}
# zfpy_kwargs = {"rate": 10}
# zfpy_kwargs = {"precision": 5}
# zfpy_kwargs = {}  # lossless reference
fill_value = -99

In [ ]:
np.random.seed(random_seed)

data = pd.DataFrame(dict(
    noise=noise_level * np.random.normal(size=(N, )),
    clean=signal_level * np.sin(np.linspace(0, signal_periods * 2 * np.pi, N)),
    valid=fraction_valid > np.random.uniform(size=(N, ))
))

# add noisy data
data["noisy"] = data["clean"] + data["noise"]

data

In [ ]:
data["noisy"].where(data["valid"]).hvplot.line()

## Scenario A

Fill in invalid data with constant values, compress whole array, reconstruct whole arary, re-mask invalid values.

In [ ]:
noisy_orig_A = data["noisy"].where(data["valid"]).fillna(fill_value)

In [ ]:
noisy_orig_A.hvplot()

In [ ]:
noisy_comp_A = zfpy.compress_numpy(noisy_orig_A.to_numpy(), **zfpy_kwargs)
rate_A = len(noisy_comp_A) / noisy_orig_A.to_numpy().nbytes
print(f"{rate_A:.2%}")
data["noisy_rec_A"] = zfpy.decompress_numpy(noisy_comp_A)
data["noisy_rec_A"] = data["noisy_rec_A"].where(data["valid"])

In [ ]:
(
    noisy_orig_A.where(data["valid"]).hvplot.line() 
    + data["noisy_rec_A"].hvplot()
    + (data["noisy_rec_A"] - noisy_orig_A).where(data["valid"]).hvplot(label="Error in reconstruction")
).cols(1)

In [ ]:
is_nan = (data["noisy_rec_A"] - noisy_orig_A).where(data["valid"]).isna()
is_smaller_than_tolerance = ((data["noisy_rec_A"] - noisy_orig_A).where(data["valid"]) < zfpy_kwargs['tolerance'])
(is_nan | is_smaller_than_tolerance).all() == True

## Scenario B

Drop invalid data, compress and reconstruct only the valid sub array, re-apply old index to get full time series.

In [ ]:
noisy_orig_B = data["noisy"].where(data["valid"])

display(noisy_orig_B.hvplot())

noisy_orig_B = noisy_orig_B.dropna()

In [ ]:
noisy_comp_B = zfpy.compress_numpy(noisy_orig_B.to_numpy(), **zfpy_kwargs)
rate_B = len(noisy_comp_B) / N / 8
print(f"{rate_B:.2%}")

In [ ]:
data["noisy_rec_B"] = zfpy.decompress_numpy(noisy_comp_B) + 0 * noisy_orig_B

In [ ]:
(
    data["noisy"].where(data["valid"]).hvplot() * data["noisy_rec_B"].hvplot()
    + (data["noisy_rec_B"] - noisy_orig_B).where(data["valid"]).hvplot()
).cols(1)

## Compare Scenarios

In [ ]:
(
    data["noisy_rec_B"].hvplot() * data["noisy_rec_A"].hvplot()
    + (data["noisy_rec_B"] - data["noisy_rec_A"]).hvplot()
    + (
        abs(data["noisy_rec_B"] - data["noisy"]).hvplot.hist(alpha=0.5, bins=20, label="B")
        * abs(data["noisy_rec_A"] - data["noisy"]).hvplot.hist(alpha=0.5, bins=20, label="A")
    )
).cols(1)

# Results

- There is no major difference between `Scenario A` and `B`. Why? Possible reasons:
  - Randomized data is difficult to be compressed.
  - Content fill_value is very important since these can be chosen related/unrelated to the data. (e.g. Range between -2,2 and fillvalue of 50)
  - `zfp` does use transformation-based compression and not prediction-based compression.
  - 1-dim vs n-dim data can show different results.

# Todo

- [ ] Test with n-dim climate/ocean model data output.
- [ ] Test with prediction-based compression model.

# Questions

- How can we relate this to AI/ML-based methods?